# Notebook for Week 3 assigment

### This notebook has as objective perform a wikipedia page scraping and created a dataframe from it

### Este notebook tem como finalidade efetuar o parser de uma página web para coletar os dados e popular dataframes

This Notebook has been produced by __Fábio Vieweger Vasques__

English: First step is import all dependecies needed to perform the web scraping.

Portuguese: Primeira coisa a ser feita é importar todas as bibliotecas necessárias para o processo de web scraping.

In [15]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import folium

Now create the dataframe where the information scraped will be stored.

In [2]:
columns = ('Postcode', 'Borough', 'Neighborhood')
df_neibors = pd.DataFrame(columns = columns)
df_neibors

,Postcode,Borough,Neighborhood


Load the data from Wikipedia

In [3]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page_response = requests.get(URL)
html_content = BeautifulSoup(page_response.content, 'html.parser')
table_content = html_content.find('table', class_= 'wikitable sortable').find('tbody').find_all('tr')
table_content

[<tr>
 <th>Postcode</th>
 <th>Borough</th>
 <th>Neighbourhood
 </th></tr>, <tr>
 <td>M1A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M2A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M3A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td></tr>, <tr>
 <td>M4A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
 </td></tr>, <tr>
 <td>M5A</td>
 <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
 <td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
 </td></tr>, <tr>
 <td>M6A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
 </td></tr>, <tr>
 <td>M6A</td>
 <td><a href="/wiki/North_Yor

Parse the data loaded from the HTML file and update the data frame.

In [4]:
for i in range(len(table_content)-1): # -1 is set because the first tag is not a td tag and it is counted on the table_content;
    linha = table_content[i+1].find_all('td') # +1 is set because the first value for i will be 0, wich is the th tag;
    Postcode = linha[0].text
    Borough = linha[1].text
    Neighborhood = linha[2].text
    
    df_neibors = df_neibors.append({'Postcode':Postcode,
                                    'Borough':Borough,
                                    'Neighborhood':Neighborhood}, ignore_index=True)

In [5]:
df_neibors.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


Now we have to work the dataframe doing the following steps:
* Remove all the rows wich contain a borough with _Not assigned_ text;
* Assign the borough values to Neighborhoods when it is _Not assigned_ ;
* Almost all of the neighborhoods have been loaded with a _/n_ value at the end of its name. Let's clean it up!
* Fourth step consist in setting our data frane with all Neighborhoods for Postcode/Borough in the same column.

In [6]:
#First we have to remove all columns wich contains "Not assigned" for the Borough.
df_neibors.drop(df_neibors.loc[df_neibors['Borough']=='Not assigned'].index, inplace=True)

#Second step is to set all the Neighborhood wich are with "Not assigned" to receive their values from Borough.
df_neibors['Neighborhood'].loc[df_neibors['Neighborhood']=='Not assigned\n'] = df_neibors['Borough']

#Third step is to remove the "\n" values found at the end of most Neighborhoods.
df_neibors['Neighborhood'] = df_neibors['Neighborhood'].map(lambda x: x.rstrip('\n'))

#Final step is the final DF with all Neighborhoods for Postcode/Borough in the same column.
df_neibors = df_neibors.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(', '.join)

df_neibors = df_neibors.reset_index()
df_neibors

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [7]:
df_neibors.shape

(103, 3)

Ok, now we have to update the dataframe adding two more columns: Latitude and Longitude. These two columns will be loaded by a different dataframe that will be defined on a second momment.

In [8]:
df_neibors['Latitude'] = ''
df_neibors['Longitude'] = ''

df_neibors.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",,
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",,
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",,
3,M1G,Scarborough,Woburn,,
4,M1H,Scarborough,Cedarbrae,,


Import data from file with coordinates from Toronto to a new dataframe

In [9]:
csv_coord = pd.read_csv("http://cocl.us/Geospatial_data")
csv_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Set both dataframes index to match their values to allow the join operation between them

In [10]:
df_neibors.set_index('Postcode', inplace=True)
csv_coord.set_index('Postal Code', inplace=True)
df_neibors['Latitude'] = csv_coord['Latitude']
df_neibors['Longitude'] = csv_coord['Longitude']

Finally load coordinates within df_neibors dataframe

In [11]:
df_neibors = df_neibors.reset_index()
df_neibors

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


Let's take a look at Toronto map from folium library

In [16]:
toronto_map = folium.Map(location=[43.6532, -79.3832], zoom_start=11)
toronto_map

And finally we will plot our dataframe locations on the map

In [17]:
for lat, lng, postcode, neighborhood in zip(df_neibors['Latitude'], df_neibors['Longitude'], df_neibors['Postcode'], df_neibors['Neighborhood']):
    label = '{}, {}'.format(postcode, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map